# Periodic download of IEX stock-ticker data

## Read IEX API credentials from `~/.config/iex.ini`

In [1]:
from pathlib import Path
config_path = Path.home() / '.config' / 'iex.ini'

from configparser import ConfigParser
config = ConfigParser()
config.read(str(config_path))
iex_config = config['iex']

api = 'https://cloud.iexapis.com'
public_key = iex_config['public_key']
secret_key = iex_config['secret_key']

In [2]:
tickers = sorted("MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AMG AFL A APD AKAM ALK ALB ARE ALXN ALGN ALLE AGN ADS LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS ANTM AON AOS APA AIV AAPL AMAT APTV ADM ARNC ANET AJG AIZ ATO T ADSK ADP AZO AVB AVY BKR BLL BAC BK BAX BBT BDX BRK.B BBY BIIB BLK HRB BA BKNG BWA BXP BSX BMY AVGO BR BF.B CHRW COG CDNS CPB COF CPRI CAH KMX CCL CAT CBOE CBRE CBS CDW CE CELG CNC CNP CTL CERN CF SCHW CHTR CVX CMG CB CHD CI XEC CINF CTAS CSCO C CFG CTXS CLX CME CMS KO CTSH CL CMCSA CMA CAG CXO COP ED STZ COO CPRT GLW CTVA COST COTY CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN FANG DLR DFS DISCA DISCK DISH DG DLTR D DOV DOW DTE DUK DRE DD DXC ETFC EMN ETN EBAY ECL EIX EW EA EMR ETR EOG EFX EQIX EQR ESS EL EVRG ES RE EXC EXPE EXPD EXR XOM FFIV FB FAST FRT FDX FIS FITB FE FRC FISV FLT FLIR FLS FMC F FTNT FTV FBHS FOXA FOX BEN FCX GPS GRMN IT GD GE GIS GM GPC GILD GL GPN GS GWW HAL HBI HOG HIG HAS HCA HCP HP HSIC HSY HES HPE HLT HFC HOLX HD HON HRL HST HPQ HUM HBAN HII IEX IDXX INFO ITW ILMN IR INTC ICE IBM INCY IP IPG IFF INTU ISRG IVZ IPGP IQV IRM JKHY JEC JBHT SJM JNJ JCI JPM JNPR KSU K KEY KEYS KMB KIM KMI KLAC KSS KHC KR LB LHX LH LRCX LW LVS LEG LDOS LEN LLY LNC LIN LKQ LMT L LOW LYB MTB MAC M MRO MPC MKTX MAR MMC MLM MAS MA MKC MXIM MCD MCK MDT MRK MET MTD MGM MCHP MU MSFT MAA MHK TAP MDLZ MNST MCO MS MOS MSI MSCI MYL NDAQ NOV NTAP NFLX NWL NEM NWSA NWS NEE NLSN NKE NI NBL JWN NSC NTRS NOC NCLH NRG NUE NVDA NVR ORLY OXY OMC OKE ORCL PCAR PKG PH PAYX PYPL PNR PBCT PEP PKI PRGO PFE PM PSX PNW PXD PNC PPG PPL PFG PG PGR PLD PRU PEG PSA PHM PVH QRVO PWR QCOM DGX RL RJF RTN O REG REGN RF RSG RMD RHI ROK ROL ROP ROST RCL CRM SBAC SLB STX SEE SRE SHW SPG SWKS SLG SNA SO LUV SPGI SWK SBUX STT SYK STI SIVB SYMC SYF SNPS SYY TMUS TROW TTWO TPR TGT TEL FTI TFX TXN TXT TMO TIF TWTR TJX TSCO TDG TRV TRIP TSN UDR ULTA USB UAA UA UNP UAL UNH UPS URI UTX UHS UNM VFC VLO VAR VTR VRSN VRSK VZ VRTX VIAB V VNO VMC WAB WMT WBA DIS WM WAT WEC WCG WFC WELL WDC WU WRK WY WHR WMB WLTW WYNN XEL XRX XLNX XYL YUM ZBH ZION ZTS".split(" "))
num_tickers = len(tickers)
num_tickers

505

In [33]:
aapl = tickers.index('AAPL'); aapl

3

In [3]:
from datetime import datetime
from dateutil.parser import parse
from datetime import timedelta as Δ

time = datetime.now
now = time()
today = now.date()
today

datetime.date(2019, 10, 30)

In [4]:
data_dir = Path.cwd() / 'data'
data_dir.mkdir(parents=True, exist_ok=True)

In [5]:
from sys import executable as python
!{python} -m pip install -Uq requests
from requests import get as GET

In [6]:
import json

def fetch(date_str, ticker):
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if out_path.exists():
        return True

    print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    with out_path.open('wb') as f:
        f.write(resp.content)

    data = json.loads(resp.content)
    if data:
        return True

    return False

In [7]:
%%time
from concurrent.futures import ThreadPoolExecutor

end_date = today
start_date = datetime(2019, 8, 1).date()
N = 32

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

dates = list(get_dates(start_date, end_date))

for date in dates:
    date_str = date.strftime('%Y%m%d')
    
    with ThreadPoolExecutor(max_workers = N) as p:
        results = p.map(lambda ticker: fetch(date_str, ticker), tickers)
    
    found_data = True in results
    if not found_data:
        print('No data found for %s; breaking' % date)

CPU times: user 1.22 s, sys: 387 ms, total: 1.61 s
Wall time: 1.36 s


In [8]:
!{python} -m pip install -Uq pandas
from pandas import DataFrame as DF, read_csv, read_json
import pandas as pd

In [9]:
minutes = 390  # [9:30am,4:00pm)

In [10]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]
cols = [ 'datetime', 'ticker' ] + features

In [11]:
def load_data_arr(date, ticker):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if not out_path.exists():
        arr = zeros((minutes, len(features)))
        arr[:] = nan
        return arr
    df = read_json(out_path)
    if df.empty:
        arr = zeros((minutes, len(features)))
        arr[:] = nan
        return arr
    arr = df[features].values
    assert arr.shape == (minutes, len(features))
    return arr

In [12]:
!{python} -m pip install -Uq numpy
import numpy as np
from numpy import array, nan, zeros

In [13]:
def load_date_arr(date):
    arr = array([ 
        load_data_arr(start_date, ticker) 
        for ticker in tickers 
    ]) \
    .reshape((
        minutes, 
        len(tickers), 
        len(features),
    ))
    assert arr.shape == (minutes, num_tickers, len(features))
    return arr

In [14]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

In [262]:
from numpy import count_nonzero as cnz, isnan as na

In [379]:
from tensorflow.keras.layers import Input

In [217]:
ticker = 'AAPL'
num_tickers = 1

In [349]:
aapl = array([ load_data_arr(date, ticker) for date in dates ]); aapl.shape

(64, 390, 8)

In [350]:
shape = aapl.shape
aapl = aapl.reshape((shape[0] * shape[1], shape[2]))
aapl = aapl[:, 4]
shape = aapl.shape; shape

(24960,)

In [351]:
y = np.roll(avgs, -1)
y = y[:-1]
x = aapl[:-1]
x.shape, y.shape

((24959,), (24959,))

In [352]:
window = 20
n = x.shape[0]
x = np.array([ x[i:(i+window)] for i in range(n-window) ])
y = y[window:]
x.shape, y.shape

((24939, 20), (24939,))

In [353]:
idxs = np.logical_and([ (cnz(na(row)) == 0) for row in x ], ~na(y))
y = y[idxs]
x = x[idxs]
x.shape, y.shape

((18463, 20), (18463,))

In [360]:
x.shape, y.shape

((18463, 20), (18463,))

In [400]:
from numpy import mean, std

In [401]:
vals = aapl[~na(aapl)]
u = np.mean(vals)
mean((vals-u)**2)

181.3834607423043

In [402]:
mean(ty), std(ty), mean((ty-mean(ty))**2)

(213.7095086060606, 10.457293365325054, 109.35498452847138)

In [362]:
n = x.shape[0]
val = 1963
trn = n - val
tx, ty = x[:trn], y[:trn]
vx, vy = x[trn:], y[trn:]
tx = tx.reshape(tx.shape + (1,))
vx = vx.reshape(vx.shape + (1,))

In [380]:
model = Sequential([
    Input(shape=(window, 1)),
    SimpleRNN(8, input_shape=(window, 1)),
    Dense(1),
])
model.build()
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_17 (SimpleRNN)    (None, 8)                 80        
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 9         
Total params: 89
Trainable params: 89
Non-trainable params: 0
_________________________________________________________________


In [381]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=40)

Train on 16500 samples, validate on 1963 samples
Epoch 1/40
16500/16500 [==============================] - 9s 560us/sample - loss: 44777.7019 - val_loss: 55744.5477
Epoch 2/40
16500/16500 [==============================] - 2s 93us/sample - loss: 43538.8645 - val_loss: 54364.1220
Epoch 3/40
16500/16500 [==============================] - 2s 92us/sample - loss: 42324.5288 - val_loss: 53008.3513
Epoch 4/40
16500/16500 [==============================] - 2s 91us/sample - loss: 41133.7890 - val_loss: 51676.5793
Epoch 5/40
16500/16500 [==============================] - 2s 94us/sample - loss: 39965.6380 - val_loss: 50367.6317
Epoch 6/40
16500/16500 [==============================] - 2s 92us/sample - loss: 38819.1267 - val_loss: 49080.4976
Epoch 7/40
16500/16500 [==============================] - 2s 105us/sample - loss: 37693.4104 - val_loss: 47814.1664
Epoch 8/40
16500/16500 [==============================] - 2s 93us/sample - loss: 36587.8074 - val_loss: 46568.1162
Epoch 9/40
16500/16500 [=====

In [383]:
vx[:10]

array([[[235.622],
        [235.671],
        [235.688],
        [235.73 ],
        [235.936],
        [236.08 ],
        [236.042],
        [236.059],
        [235.961],
        [236.   ],
        [235.96 ],
        [235.901],
        [235.8  ],
        [235.785],
        [235.82 ],
        [235.759],
        [235.761],
        [235.7  ],
        [235.585],
        [235.639]],

       [[235.671],
        [235.688],
        [235.73 ],
        [235.936],
        [236.08 ],
        [236.042],
        [236.059],
        [235.961],
        [236.   ],
        [235.96 ],
        [235.901],
        [235.8  ],
        [235.785],
        [235.82 ],
        [235.759],
        [235.761],
        [235.7  ],
        [235.585],
        [235.639],
        [235.735]],

       [[235.688],
        [235.73 ],
        [235.936],
        [236.08 ],
        [236.042],
        [236.059],
        [235.961],
        [236.   ],
        [235.96 ],
        [235.901],
        [235.8  ],
        [235.785],
        

In [382]:
model.predict(vx[:10])

array([[113.766594],
       [113.766594],
       [113.766594],
       [113.766594],
       [113.766594],
       [113.766594],
       [113.766594],
       [113.766594],
       [113.766594],
       [113.766594]], dtype=float32)

In [384]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=40)

Train on 16500 samples, validate on 1963 samples
Epoch 1/40
16500/16500 [==============================] - 2s 98us/sample - loss: 9828.1939 - val_loss: 15249.4345
Epoch 2/40
16500/16500 [==============================] - 2s 94us/sample - loss: 9297.5158 - val_loss: 14583.9656
Epoch 3/40
16500/16500 [==============================] - 2s 92us/sample - loss: 8782.8788 - val_loss: 13934.8672
Epoch 4/40
16500/16500 [==============================] - 2s 98us/sample - loss: 8284.2093 - val_loss: 13302.0760
Epoch 5/40
16500/16500 [==============================] - 2s 98us/sample - loss: 7801.5583 - val_loss: 12685.5464
Epoch 6/40
16500/16500 [==============================] - 2s 106us/sample - loss: 7334.7564 - val_loss: 12085.2790
Epoch 7/40
16500/16500 [==============================] - 2s 109us/sample - loss: 6883.8133 - val_loss: 11501.3407
Epoch 8/40
16500/16500 [==============================] - 2s 103us/sample - loss: 6448.6567 - val_loss: 10933.4256
Epoch 9/40
16500/16500 [============

In [386]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=40)

Train on 16500 samples, validate on 1963 samples
Epoch 1/40
16500/16500 [==============================] - 2s 101us/sample - loss: 126.5531 - val_loss: 905.4306
Epoch 2/40
16500/16500 [==============================] - 2s 98us/sample - loss: 118.7578 - val_loss: 847.4617
Epoch 3/40
16500/16500 [==============================] - 2s 98us/sample - loss: 114.0279 - val_loss: 801.1946
Epoch 4/40
16500/16500 [==============================] - 2s 101us/sample - loss: 111.4415 - val_loss: 766.9980
Epoch 5/40
16500/16500 [==============================] - 2s 96us/sample - loss: 110.1865 - val_loss: 742.2547
Epoch 6/40
16500/16500 [==============================] - 2s 97us/sample - loss: 109.6367 - val_loss: 726.3992
Epoch 7/40
16500/16500 [==============================] - 2s 97us/sample - loss: 109.4357 - val_loss: 716.1499
Epoch 8/40
16500/16500 [==============================] - 2s 96us/sample - loss: 109.3782 - val_loss: 710.8648
Epoch 9/40
16500/16500 [==============================] - 2s 

In [387]:
model.predict(vx[:10])

array([[213.7478],
       [213.7478],
       [213.7478],
       [213.7478],
       [213.7478],
       [213.7478],
       [213.7478],
       [213.7478],
       [213.7478],
       [213.7478]], dtype=float32)

In [388]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=100)

Train on 16500 samples, validate on 1963 samples
Epoch 1/100
16500/16500 [==============================] - 2s 102us/sample - loss: 109.3671 - val_loss: 707.2414
Epoch 2/100
16500/16500 [==============================] - 2s 92us/sample - loss: 109.3646 - val_loss: 710.0116
Epoch 3/100
16500/16500 [==============================] - 2s 94us/sample - loss: 109.3631 - val_loss: 703.9060
Epoch 4/100
16500/16500 [==============================] - 2s 93us/sample - loss: 109.3627 - val_loss: 706.1498
Epoch 5/100
16500/16500 [==============================] - 2s 99us/sample - loss: 109.3648 - val_loss: 706.6095
Epoch 6/100
16500/16500 [==============================] - 2s 109us/sample - loss: 109.3588 - val_loss: 703.1959
Epoch 7/100
16500/16500 [==============================] - 2s 107us/sample - loss: 109.3603 - val_loss: 708.7431
Epoch 8/100
16500/16500 [==============================] - 2s 109us/sample - loss: 109.3650 - val_loss: 707.7551
Epoch 9/100
16500/16500 [==========================

Epoch 73/100
16500/16500 [==============================] - 2s 95us/sample - loss: 109.3639 - val_loss: 705.6766
Epoch 74/100
16500/16500 [==============================] - 2s 107us/sample - loss: 109.3636 - val_loss: 706.0243
Epoch 75/100
16500/16500 [==============================] - 2s 95us/sample - loss: 109.3688 - val_loss: 704.2413
Epoch 76/100
16500/16500 [==============================] - 2s 104us/sample - loss: 109.3662 - val_loss: 707.5278
Epoch 77/100
16500/16500 [==============================] - 2s 99us/sample - loss: 109.3722 - val_loss: 708.1690
Epoch 78/100
16500/16500 [==============================] - 2s 95us/sample - loss: 109.3652 - val_loss: 710.3451
Epoch 79/100
16500/16500 [==============================] - 2s 97us/sample - loss: 109.3654 - val_loss: 704.8689
Epoch 80/100
16500/16500 [==============================] - 2s 102us/sample - loss: 109.3664 - val_loss: 706.5040
Epoch 81/100
16500/16500 [==============================] - 2s 101us/sample - loss: 109.3671 

In [390]:
p = model.predict(vx)
p.max() - p.min()

0.0

In [392]:
p[:10]

array([[213.6402],
       [213.6402],
       [213.6402],
       [213.6402],
       [213.6402],
       [213.6402],
       [213.6402],
       [213.6402],
       [213.6402],
       [213.6402]], dtype=float32)

In [391]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=100)

Train on 16500 samples, validate on 1963 samples
Epoch 1/100
16500/16500 [==============================] - 2s 98us/sample - loss: 109.3669 - val_loss: 709.2678
Epoch 2/100
16500/16500 [==============================] - 2s 94us/sample - loss: 109.3687 - val_loss: 707.0782
Epoch 3/100
16500/16500 [==============================] - 2s 108us/sample - loss: 109.3663 - val_loss: 708.5653
Epoch 4/100
16500/16500 [==============================] - 2s 115us/sample - loss: 109.3642 - val_loss: 708.9313
Epoch 5/100
16500/16500 [==============================] - 2s 111us/sample - loss: 109.3600 - val_loss: 708.9434
Epoch 6/100
16500/16500 [==============================] - 2s 103us/sample - loss: 109.3682 - val_loss: 710.4293
Epoch 7/100
16500/16500 [==============================] - 2s 100us/sample - loss: 109.3567 - val_loss: 702.3865
Epoch 8/100
16500/16500 [==============================] - 2s 92us/sample - loss: 109.3679 - val_loss: 705.2499
Epoch 9/100
16500/16500 [=========================

Epoch 73/100
16500/16500 [==============================] - 2s 95us/sample - loss: 109.3618 - val_loss: 708.9562
Epoch 74/100
16500/16500 [==============================] - 2s 98us/sample - loss: 109.3672 - val_loss: 706.0019
Epoch 75/100
16500/16500 [==============================] - 2s 108us/sample - loss: 109.3614 - val_loss: 705.7733
Epoch 76/100
16500/16500 [==============================] - 2s 96us/sample - loss: 109.3652 - val_loss: 701.3019
Epoch 77/100
16500/16500 [==============================] - 2s 94us/sample - loss: 109.3733 - val_loss: 705.3530
Epoch 78/100
16500/16500 [==============================] - 2s 94us/sample - loss: 109.3631 - val_loss: 711.0140
Epoch 79/100
16500/16500 [==============================] - 2s 96us/sample - loss: 109.3685 - val_loss: 707.5135
Epoch 80/100
16500/16500 [==============================] - 2s 108us/sample - loss: 109.3637 - val_loss: 704.8928
Epoch 81/100
16500/16500 [==============================] - 2s 108us/sample - loss: 109.3640 -

In [393]:
p = model.predict(vx)
p.max() - p.min(), p[:10]

(0.0, array([[213.73215],
        [213.73215],
        [213.73215],
        [213.73215],
        [213.73215],
        [213.73215],
        [213.73215],
        [213.73215],
        [213.73215],
        [213.73215]], dtype=float32))

In [394]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=100)

p = model.predict(vx)
p.max() - p.min(), p[:10]

Train on 16500 samples, validate on 1963 samples
Epoch 1/100
16500/16500 [==============================] - 2s 101us/sample - loss: 109.3682 - val_loss: 706.7606
Epoch 2/100
16500/16500 [==============================] - 2s 98us/sample - loss: 109.3673 - val_loss: 706.7103
Epoch 3/100
16500/16500 [==============================] - 2s 95us/sample - loss: 109.3727 - val_loss: 705.2355
Epoch 4/100
16500/16500 [==============================] - 2s 98us/sample - loss: 109.3662 - val_loss: 707.4446
Epoch 5/100
16500/16500 [==============================] - 2s 97us/sample - loss: 109.3674 - val_loss: 707.8880
Epoch 6/100
16500/16500 [==============================] - 2s 107us/sample - loss: 109.3634 - val_loss: 709.1869
Epoch 7/100
16500/16500 [==============================] - 2s 94us/sample - loss: 109.3665 - val_loss: 708.9145
Epoch 8/100
16500/16500 [==============================] - 2s 94us/sample - loss: 109.3632 - val_loss: 708.2234
Epoch 9/100
16500/16500 [============================

Epoch 73/100
16500/16500 [==============================] - 2s 109us/sample - loss: 109.3683 - val_loss: 706.1058
Epoch 74/100
16500/16500 [==============================] - 2s 95us/sample - loss: 109.3727 - val_loss: 704.5431
Epoch 75/100
16500/16500 [==============================] - 2s 96us/sample - loss: 109.3668 - val_loss: 706.2089
Epoch 76/100
16500/16500 [==============================] - 2s 95us/sample - loss: 109.3623 - val_loss: 705.6750
Epoch 77/100
16500/16500 [==============================] - 2s 93us/sample - loss: 109.3653 - val_loss: 708.9994
Epoch 78/100
16500/16500 [==============================] - 2s 92us/sample - loss: 109.3672 - val_loss: 707.5423
Epoch 79/100
16500/16500 [==============================] - 2s 94us/sample - loss: 109.3675 - val_loss: 706.8422
Epoch 80/100
16500/16500 [==============================] - 2s 106us/sample - loss: 109.3688 - val_loss: 706.6167
Epoch 81/100
16500/16500 [==============================] - 2s 95us/sample - loss: 109.3703 - 

(0.0, array([[213.75345],
        [213.75345],
        [213.75345],
        [213.75345],
        [213.75345],
        [213.75345],
        [213.75345],
        [213.75345],
        [213.75345],
        [213.75345]], dtype=float32))

In [365]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 2s 102us/sample - loss: 23695.8891 - val_loss: 31833.5312
Epoch 2/20
16500/16500 [==============================] - 2s 105us/sample - loss: 22840.4573 - val_loss: 30839.3250
Epoch 3/20
16500/16500 [==============================] - 2s 106us/sample - loss: 22001.7827 - val_loss: 29861.9263
Epoch 4/20
16500/16500 [==============================] - 2s 103us/sample - loss: 21179.6795 - val_loss: 28901.3037
Epoch 5/20
16500/16500 [==============================] - 2s 105us/sample - loss: 20374.2148 - val_loss: 27957.5544
Epoch 6/20
16500/16500 [==============================] - 2s 105us/sample - loss: 19585.3680 - val_loss: 27030.4896
Epoch 7/20
16500/16500 [==============================] - 2s 107us/sample - loss: 18813.0083 - val_loss: 26119.8637
Epoch 8/20
16500/16500 [==============================] - 2s 100us/sample - loss: 18057.2038 - val_loss: 25226.3443
Epoch 9/20
16500/16500 

In [366]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 2s 113us/sample - loss: 9723.0399 - val_loss: 15117.8777
Epoch 2/20
16500/16500 [==============================] - 2s 101us/sample - loss: 9195.5388 - val_loss: 14455.6754
Epoch 3/20
16500/16500 [==============================] - 2s 108us/sample - loss: 8684.0220 - val_loss: 13809.6919
Epoch 4/20
16500/16500 [==============================] - 2s 101us/sample - loss: 8188.4706 - val_loss: 13180.2714
Epoch 5/20
16500/16500 [==============================] - 2s 100us/sample - loss: 7708.9248 - val_loss: 12566.7594
Epoch 6/20
16500/16500 [==============================] - 2s 110us/sample - loss: 7245.3244 - val_loss: 11969.7513
Epoch 7/20
16500/16500 [==============================] - 2s 99us/sample - loss: 6797.5033 - val_loss: 11388.8811
Epoch 8/20
16500/16500 [==============================] - 2s 98us/sample - loss: 6365.3931 - val_loss: 10824.2146
Epoch 9/20
16500/16500 [=========

In [367]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 2s 111us/sample - loss: 2151.3395 - val_loss: 4937.0624
Epoch 2/20
16500/16500 [==============================] - 2s 111us/sample - loss: 1931.6576 - val_loss: 4594.1900
Epoch 3/20
16500/16500 [==============================] - 2s 102us/sample - loss: 1726.2325 - val_loss: 4266.5968
Epoch 4/20
16500/16500 [==============================] - 2s 100us/sample - loss: 1534.9350 - val_loss: 3954.3587
Epoch 5/20
16500/16500 [==============================] - 2s 105us/sample - loss: 1357.5372 - val_loss: 3657.3540
Epoch 6/20
16500/16500 [==============================] - 2s 101us/sample - loss: 1193.8296 - val_loss: 3375.6710
Epoch 7/20
16500/16500 [==============================] - 2s 98us/sample - loss: 1043.5504 - val_loss: 3108.9675
Epoch 8/20
16500/16500 [==============================] - 2s 98us/sample - loss: 906.4682 - val_loss: 2857.1506
Epoch 9/20
16500/16500 [==================

In [368]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 2s 103us/sample - loss: 124.7279 - val_loss: 893.0323
Epoch 2/20
16500/16500 [==============================] - 2s 101us/sample - loss: 117.6009 - val_loss: 837.3855
Epoch 3/20
16500/16500 [==============================] - 2s 106us/sample - loss: 113.3840 - val_loss: 793.8091
Epoch 4/20
16500/16500 [==============================] - 2s 100us/sample - loss: 111.1027 - val_loss: 761.3085
Epoch 5/20
16500/16500 [==============================] - 2s 102us/sample - loss: 110.0161 - val_loss: 737.6249
Epoch 6/20
16500/16500 [==============================] - 2s 111us/sample - loss: 109.5734 - val_loss: 723.0645
Epoch 7/20
16500/16500 [==============================] - 2s 100us/sample - loss: 109.4179 - val_loss: 715.2676
Epoch 8/20
16500/16500 [==============================] - 2s 100us/sample - loss: 109.3765 - val_loss: 710.4165
Epoch 9/20
16500/16500 [==============================]

In [370]:
w = model.get_weights()
[ r.shape for r in w ]

[(1, 8), (8, 8), (8,), (8, 1), (1,)]

In [371]:
w

[array([[-0.5911007 , -0.09386152,  0.7858523 ,  0.13839978, -0.04129133,
         -0.1983273 ,  0.19056714,  0.12164313]], dtype=float32),
 array([[ 0.66170406,  0.07864968, -0.0857741 ,  0.00465569,  0.23122407,
          0.23979928,  0.29800957, -0.6132319 ],
        [ 0.10043948,  0.43177623,  0.54822636,  0.04553895, -0.08450262,
          0.6372433 , -0.10235932,  0.24625379],
        [-0.18622677,  0.424772  , -0.18163659,  0.07859045, -0.08238811,
         -0.03797168, -0.71313715, -0.48532724],
        [-0.20455588, -0.30253392, -0.22140023, -0.40680477, -0.7330993 ,
          0.3933699 ,  0.03625377, -0.23546964],
        [-0.01476307,  0.21162851,  0.42626294, -0.7794128 ,  0.11725012,
         -0.36578918,  0.05476402, -0.14893378],
        [ 0.52338564,  0.25230432, -0.4987298 , -0.30956528, -0.07566322,
         -0.06359223, -0.22296605,  0.49302447],
        [-0.19393167,  0.61659366, -0.1611582 ,  0.20718573, -0.4673722 ,
         -0.25404182,  0.53650063, -0.04978885],

In [375]:
model.evaluate(vx[:1], vy[:1], batch_size=1)

1/1 [==============================] - 0s 7ms/sample - loss: 487.4119


487.41192626953125

In [377]:
vx[:10]

array([[[235.622],
        [235.671],
        [235.688],
        [235.73 ],
        [235.936],
        [236.08 ],
        [236.042],
        [236.059],
        [235.961],
        [236.   ],
        [235.96 ],
        [235.901],
        [235.8  ],
        [235.785],
        [235.82 ],
        [235.759],
        [235.761],
        [235.7  ],
        [235.585],
        [235.639]],

       [[235.671],
        [235.688],
        [235.73 ],
        [235.936],
        [236.08 ],
        [236.042],
        [236.059],
        [235.961],
        [236.   ],
        [235.96 ],
        [235.901],
        [235.8  ],
        [235.785],
        [235.82 ],
        [235.759],
        [235.761],
        [235.7  ],
        [235.585],
        [235.639],
        [235.735]],

       [[235.688],
        [235.73 ],
        [235.936],
        [236.08 ],
        [236.042],
        [236.059],
        [235.961],
        [236.   ],
        [235.96 ],
        [235.901],
        [235.8  ],
        [235.785],
        

In [378]:
model.predict(vx[:10])

array([[213.66759],
       [213.66759],
       [213.66759],
       [213.66759],
       [213.66759],
       [213.66759],
       [213.66759],
       [213.66759],
       [213.66759],
       [213.66759]], dtype=float32)

In [373]:
model.evaluate(vx, vy, batch_size=50)

1963/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

708.4187838787093

# AAPL-only

In [316]:
shape = aapl.shape
aapl = aapl.reshape((shape[0] * shape[1], shape[2]))
shape = aapl.shape; shape

(24960, 8)

In [317]:
avgs = aapl[:, 4]; avgs

array([214.718, 215.417, 215.43 , ..., 243.282, 243.258, 243.297])

In [318]:
y = np.roll(avgs, -1)
y = y[:-1]
x = aapl[:-1]
x.shape, y.shape

((24959, 8), (24959,))

In [319]:
window = 20
n = x.shape[0]
x = np.array([ x[i:(i+window)] for i in range(n-window) ])
y = y[window:]
x.shape, y.shape

((24939, 20, 8), (24939,))

In [320]:
idxs = np.logical_and([ (cnz(na(row)) == 0) for row in x ], ~na(y))
y = y[idxs]
x = x[idxs]
x.shape, y.shape

((18463, 20, 8), (18463,))

In [321]:
n = x.shape[0]
val = 1963
trn = n - val
tx, ty = x[:trn], y[:trn]
vx, vy = x[trn:], y[trn:]

In [333]:
x[:,-1,4].shape, y.shape

((18463,), (18463,))

In [337]:
y

array([214.435, 214.594, 214.476, ..., 243.282, 243.258, 243.297])

In [336]:
d

array([-0.059, -0.289, -0.041, ...,  0.202,  0.161, -0.015])

In [338]:
d = x[:,-1,4] - y
np.mean(d**2)

0.06451057146725886

In [322]:
[ a.shape for a in [tx,ty,vx,vy] ]

[(16500, 20, 8), (16500,), (1963, 20, 8), (1963,)]

In [340]:
from tensorflow.keras.layers import LSTM

In [341]:
model = Sequential([
    LSTM(32, input_shape=(window, len(features))),
    Dense(1),
])
model.build()
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5248      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 33        
Total params: 5,281
Trainable params: 5,281
Non-trainable params: 0
_________________________________________________________________


In [342]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 5s 308us/sample - loss: 44988.6397 - val_loss: 55906.6671
Epoch 2/20
16500/16500 [==============================] - 3s 171us/sample - loss: 43137.6257 - val_loss: 53393.1119
Epoch 3/20
16500/16500 [==============================] - 3s 175us/sample - loss: 41233.0813 - val_loss: 51560.3246
Epoch 4/20
16500/16500 [==============================] - 3s 179us/sample - loss: 39622.2297 - val_loss: 49728.6806
Epoch 5/20
16500/16500 [==============================] - 3s 174us/sample - loss: 37480.2085 - val_loss: 46678.4399
Epoch 6/20
16500/16500 [==============================] - 3s 170us/sample - loss: 35155.7298 - val_loss: 44440.6561
Epoch 7/20
16500/16500 [==============================] - 3s 172us/sample - loss: 33338.3102 - val_loss: 42551.0105
Epoch 8/20
16500/16500 [==============================] - 3s 180us/sample - loss: 31739.7855 - val_loss: 40770.5517
Epoch 9/20
16500/16500 

In [343]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 3s 196us/sample - loss: 15612.2131 - val_loss: 22132.6820
Epoch 2/20
16500/16500 [==============================] - 3s 177us/sample - loss: 14636.8450 - val_loss: 20966.6546
Epoch 3/20
16500/16500 [==============================] - 3s 169us/sample - loss: 13696.1606 - val_loss: 19835.6729
Epoch 4/20
16500/16500 [==============================] - 3s 168us/sample - loss: 12789.9664 - val_loss: 18739.8554
Epoch 5/20
16500/16500 [==============================] - 3s 167us/sample - loss: 11918.1591 - val_loss: 17679.0931
Epoch 6/20
16500/16500 [==============================] - 3s 189us/sample - loss: 11080.4502 - val_loss: 16653.1755
Epoch 7/20
16500/16500 [==============================] - 3s 164us/sample - loss: 10276.5481 - val_loss: 15661.6208
Epoch 8/20
16500/16500 [==============================] - 3s 161us/sample - loss: 9506.3863 - val_loss: 14704.9840
Epoch 9/20
16500/16500 [

In [344]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 3s 175us/sample - loss: 2454.3287 - val_loss: 5327.2482
Epoch 2/20
16500/16500 [==============================] - 3s 177us/sample - loss: 2128.3118 - val_loss: 4833.9684
Epoch 3/20
16500/16500 [==============================] - 3s 178us/sample - loss: 1830.9780 - val_loss: 4372.2482
Epoch 4/20
16500/16500 [==============================] - 3s 159us/sample - loss: 1561.7360 - val_loss: 3941.0871
Epoch 5/20
16500/16500 [==============================] - 3s 161us/sample - loss: 1319.8299 - val_loss: 3540.4152
Epoch 6/20
16500/16500 [==============================] - 3s 167us/sample - loss: 1104.4000 - val_loss: 3169.9071
Epoch 7/20
16500/16500 [==============================] - 3s 174us/sample - loss: 914.5995 - val_loss: 2829.1018
Epoch 8/20
16500/16500 [==============================] - 3s 161us/sample - loss: 749.3585 - val_loss: 2516.5574
Epoch 9/20
16500/16500 [=================

In [345]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 3s 183us/sample - loss: 109.7586 - val_loss: 729.1833
Epoch 2/20
16500/16500 [==============================] - 3s 166us/sample - loss: 109.4629 - val_loss: 717.4735
Epoch 3/20
16500/16500 [==============================] - 3s 163us/sample - loss: 109.3833 - val_loss: 711.8775
Epoch 4/20
16500/16500 [==============================] - 3s 161us/sample - loss: 109.3683 - val_loss: 708.9282
Epoch 5/20
16500/16500 [==============================] - 3s 168us/sample - loss: 109.3620 - val_loss: 706.8166
Epoch 6/20
16500/16500 [==============================] - 3s 169us/sample - loss: 109.3614 - val_loss: 707.7023
Epoch 7/20
16500/16500 [==============================] - 3s 165us/sample - loss: 109.3625 - val_loss: 708.9257
Epoch 8/20
16500/16500 [==============================] - 3s 162us/sample - loss: 109.3665 - val_loss: 706.3144
Epoch 9/20
16500/16500 [==============================]

In [327]:
model = Sequential([
    SimpleRNN(32, input_shape=(window, len(features))),
    Dense(1),
])
model.build()
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_14 (SimpleRNN)    (None, 32)                1312      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 33        
Total params: 1,345
Trainable params: 1,345
Non-trainable params: 0
_________________________________________________________________


In [328]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 3s 205us/sample - loss: 43744.4861 - val_loss: 52791.8456
Epoch 2/20
16500/16500 [==============================] - 2s 109us/sample - loss: 39417.0714 - val_loss: 48090.3541
Epoch 3/20
16500/16500 [==============================] - 2s 135us/sample - loss: 35407.1841 - val_loss: 43707.5682
Epoch 4/20
16500/16500 [==============================] - 2s 108us/sample - loss: 31693.2392 - val_loss: 39619.5789
Epoch 5/20
16500/16500 [==============================] - 2s 110us/sample - loss: 28255.7276 - val_loss: 35809.1404
Epoch 6/20
16500/16500 [==============================] - 2s 110us/sample - loss: 25077.8794 - val_loss: 32258.6856
Epoch 7/20
16500/16500 [==============================] - 2s 109us/sample - loss: 22145.1686 - val_loss: 28954.1052
Epoch 8/20
16500/16500 [==============================] - 2s 111us/sample - loss: 19444.3579 - val_loss: 25884.8197
Epoch 9/20
16500/16500 

In [329]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 2s 111us/sample - loss: 1322.8062 - val_loss: 3439.4641
Epoch 2/20
16500/16500 [==============================] - 2s 117us/sample - loss: 943.3676 - val_loss: 2805.0609
Epoch 3/20
16500/16500 [==============================] - 2s 116us/sample - loss: 657.7758 - val_loss: 2283.8627
Epoch 4/20
16500/16500 [==============================] - 2s 106us/sample - loss: 452.0818 - val_loss: 1864.7974
Epoch 5/20
16500/16500 [==============================] - 2s 108us/sample - loss: 312.0048 - val_loss: 1536.3445
Epoch 6/20
16500/16500 [==============================] - 2s 109us/sample - loss: 223.0949 - val_loss: 1287.4983
Epoch 7/20
16500/16500 [==============================] - 2s 109us/sample - loss: 171.1751 - val_loss: 1104.2713
Epoch 8/20
16500/16500 [==============================] - 2s 122us/sample - loss: 143.9120 - val_loss: 975.7463
Epoch 9/20
16500/16500 [=======================

In [323]:
model = Sequential([
    SimpleRNN(32, input_shape=(window, len(features))),
    Dense(1),
])
model.build()
model.compile(loss='mse', optimizer='sgd')
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_13 (SimpleRNN)    (None, 32)                1312      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 1,345
Trainable params: 1,345
Non-trainable params: 0
_________________________________________________________________


In [324]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=1)

Train on 16500 samples, validate on 1963 samples
16500/16500 [==============================] - 3s 175us/sample - loss: 290.1315 - val_loss: 720.6219


In [325]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=10)

Train on 16500 samples, validate on 1963 samples
Epoch 1/10
16500/16500 [==============================] - 2s 119us/sample - loss: 120.7651 - val_loss: 744.7262
Epoch 2/10
16500/16500 [==============================] - 2s 104us/sample - loss: 117.0561 - val_loss: 790.6062
Epoch 3/10
16500/16500 [==============================] - 2s 109us/sample - loss: 114.3773 - val_loss: 809.7257
Epoch 4/10
16500/16500 [==============================] - 2s 110us/sample - loss: 113.1029 - val_loss: 642.6632
Epoch 5/10
16500/16500 [==============================] - 2s 115us/sample - loss: 111.9965 - val_loss: 654.1763
Epoch 6/10
16500/16500 [==============================] - 2s 113us/sample - loss: 111.4130 - val_loss: 774.6058
Epoch 7/10
16500/16500 [==============================] - 2s 125us/sample - loss: 111.1374 - val_loss: 697.1080
Epoch 8/10
16500/16500 [==============================] - 2s 123us/sample - loss: 110.8589 - val_loss: 682.0563
Epoch 9/10
16500/16500 [==============================]

In [326]:
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=50,
          epochs=20)

Train on 16500 samples, validate on 1963 samples
Epoch 1/20
16500/16500 [==============================] - 2s 120us/sample - loss: 110.7725 - val_loss: 691.8990
Epoch 2/20
16500/16500 [==============================] - 2s 121us/sample - loss: 110.1673 - val_loss: 620.4467
Epoch 3/20
16500/16500 [==============================] - 2s 124us/sample - loss: 110.5189 - val_loss: 718.1927
Epoch 4/20
16500/16500 [==============================] - 2s 121us/sample - loss: 110.1622 - val_loss: 682.8046
Epoch 5/20
16500/16500 [==============================] - 2s 117us/sample - loss: 110.4196 - val_loss: 765.1008
Epoch 6/20
16500/16500 [==============================] - 2s 114us/sample - loss: 110.3792 - val_loss: 675.9035
Epoch 7/20
16500/16500 [==============================] - 2s 111us/sample - loss: 110.5333 - val_loss: 632.6029
Epoch 8/20
16500/16500 [==============================] - 2s 122us/sample - loss: 110.2806 - val_loss: 764.4253
Epoch 9/20
16500/16500 [==============================]

# Multi-ticker attempt

In [15]:
all = \
    array(
        Parallel(n_jobs=8)(delayed(load_date_arr)(date) for date in dates)
    )

In [195]:
shape[0]

24960

In [16]:
shape =  all.shape; shape

(64, 390, 505, 8)

In [17]:
all = all.reshape(shape[0] * shape[1], *shape[2:]); all.shape

(24960, 505, 8)

In [24]:
np.save(str(data_dir / 'all.npy'), all)

In [28]:
shape = all.shape
all = all.reshape(shape[:-2] + ((shape[-2] * shape[-1],)))
shape = all.shape
shape

(24960, 4040)

In [193]:
np.count_nonzero(np.isnan(all[:, aapl_avg]))

9792

In [200]:
all = all[~np.isnan(all[:, aapl_avg])]
shape = all.shape; shape

(15168, 4040)

In [202]:
aapl_avg = aapl * len(features) + 4
out = all[:, aapl_avg]
out = np.roll(out, -1)[:-1]
all = all[:-1]
shape = all.shape
out.shape, out

((15167,), array([ 91.835, 204.915,  85.943, ...,  61.1  ,  54.221, 138.386]))

In [63]:
from numpy.random import randint, shuffle
from numpy import arange, nan_to_num

In [203]:
time_steps = 20

def make_train_val_idxs(train=None, val=None):
    num_samples = shape[0] - time_steps

    if train is None and val is None:
        raise Exception('Specify at least one of {train, val}')
            
    if train is not None and train < 1:
        train = int(train * num_samples)

    if val is not None and val < 1:
        val = int(train * num_samples)
    
    if val is None:
        val = num_samples - train
    
    if train is None:
        train = num_samples - val
    
    if train + val > num_samples:
        raise Exception('%d + %d > %d' % (train, val, num_samples))

    sample_idxs = arange(time_steps, shape[0])
    shuffle(sample_idxs)
    training_idxs = sample_idxs[:train]
    validation_idxs = sample_idxs[-val:]
    return (training_idxs, validation_idxs)

def make_train_val_sets(train=None, val=None):
    train_idxs, val_idxs = make_train_val_idxs(train, val)
    train_x = nan_to_num(array([ all[(idx - time_steps):idx] for idx in train_idxs ]))
    train_y = nan_to_num(array([ all[idx][aapl_avg] for idx in train_idxs ]))
    val_x = nan_to_num(array([ all[(idx - time_steps):idx] for idx in val_idxs ]))
    val_y = nan_to_num(array([ all[idx][aapl_avg] for idx in val_idxs ]))
    return (
        train_x,
        train_y,
        val_x,
        val_y,
    )

In [204]:
train_x, train_y, val_x, val_y = make_train_val_sets(12000, 1000)
[ a.shape for a in [train_x, train_y, val_x, val_y]]

[(12000, 20, 4040), (12000,), (1000, 20, 4040), (1000,)]

In [205]:
shape[0]

15167

In [19]:
!{python} -m pip install -Uq tensorflow
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [206]:
model = Sequential([
    SimpleRNN(32, input_shape=(time_steps, num_tickers * len(features))),
    Dense(1),
])

In [207]:
model.build()

In [208]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_7 (SimpleRNN)     (None, 32)                130336    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 130,369
Trainable params: 130,369
Non-trainable params: 0
_________________________________________________________________


In [209]:
model.compile(loss='mse', optimizer='sgd')

In [210]:
model.fit(train_x, train_y, 
          #validation_data=(val_x, val_y),
          batch_size=40,
          epochs=10)

Train on 12000 samples
Epoch 1/10
12000/12000 [==============================] - 18s 1ms/sample - loss: 16344.0626
Epoch 2/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15546.2652
Epoch 3/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15530.9598
Epoch 4/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15515.1891
Epoch 5/10
12000/12000 [==============================] - 15s 1ms/sample - loss: 15502.5735
Epoch 6/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15523.4275
Epoch 7/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15517.6569
Epoch 8/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15560.4271
Epoch 9/10
12000/12000 [==============================] - 16s 1ms/sample - loss: 15538.1058
Epoch 10/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15486.1096


In [211]:
model.fit(train_x, train_y, 
          #validation_data=(val_x, val_y),
          batch_size=40,
          epochs=20)

Train on 12000 samples
Epoch 1/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15545.4982
Epoch 2/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15530.5658
Epoch 3/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15478.8371
Epoch 4/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15528.6299
Epoch 5/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15508.9193
Epoch 6/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15532.5590
Epoch 7/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15535.9591
Epoch 8/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15509.6757
Epoch 9/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15512.1479
Epoch 10/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15517.8358
Epoch 11/20
12000/12000 [==============================]

In [212]:
model.compile(loss='mse', optimizer='adam')

In [213]:
model.fit(train_x, train_y, 
          validation_data=(val_x, val_y),
          batch_size=40,
          epochs=20)

Train on 12000 samples, validate on 1000 samples
Epoch 1/20
12000/12000 [==============================] - 19s 2ms/sample - loss: 15419.7773 - val_loss: 15587.0837
Epoch 2/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15412.6172 - val_loss: 15572.2574
Epoch 3/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15406.8898 - val_loss: 15560.0918
Epoch 4/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15402.4290 - val_loss: 15549.7905
Epoch 5/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15398.7806 - val_loss: 15541.3923
Epoch 6/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15395.8807 - val_loss: 15533.2663
Epoch 7/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15393.4201 - val_loss: 15527.3017
Epoch 8/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15391.4413 - val_loss: 15521.6040
Epoch 9/20
12000/12000 [=======

In [214]:
model.fit(train_x, train_y, 
          validation_data=(val_x, val_y),
          batch_size=40,
          epochs=20)

Train on 12000 samples, validate on 1000 samples
Epoch 1/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15379.3563 - val_loss: 15491.4467
Epoch 2/20
12000/12000 [==============================] - 17s 1ms/sample - loss: 15378.7538 - val_loss: 15490.7604
Epoch 3/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15378.3939 - val_loss: 15489.9805
Epoch 4/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15377.8064 - val_loss: 15489.4879
Epoch 5/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15377.2507 - val_loss: 15488.4293
Epoch 6/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15376.9178 - val_loss: 15488.1877
Epoch 7/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15376.4122 - val_loss: 15488.0419
Epoch 8/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15375.9355 - val_loss: 15487.4029
Epoch 9/20
12000/12000 [=======

In [215]:
model.fit(train_x, train_y, 
          validation_data=(val_x, val_y),
          batch_size=40,
          epochs=200)

Train on 12000 samples, validate on 1000 samples
Epoch 1/200
12000/12000 [==============================] - 17s 1ms/sample - loss: 15371.6789 - val_loss: 15484.1590
Epoch 2/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15371.4278 - val_loss: 15484.0324
Epoch 3/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15371.1502 - val_loss: 15484.1951
Epoch 4/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.8957 - val_loss: 15483.6107
Epoch 5/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.7880 - val_loss: 15483.7685
Epoch 6/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.4712 - val_loss: 15483.7804
Epoch 7/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.2009 - val_loss: 15484.2235
Epoch 8/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.0815 - val_loss: 15483.6573
Epoch 9/200
12000/12000

Epoch 140/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15363.4934 - val_loss: 15494.7523
Epoch 141/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15363.3948 - val_loss: 15494.3557
Epoch 142/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15363.4620 - val_loss: 15495.2337
Epoch 143/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15363.4224 - val_loss: 15494.8069
Epoch 144/200
12000/12000 [==============================] - 17s 1ms/sample - loss: 15363.4144 - val_loss: 15494.9970
Epoch 145/200
12000/12000 [==============================] - 17s 1ms/sample - loss: 15363.4223 - val_loss: 15495.0199
Epoch 146/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15363.4744 - val_loss: 15495.5698
Epoch 147/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15363.3751 - val_loss: 15494.8303
Epoch 148/200
12000/12000 [=============================

In [176]:
model.fit(train_x, train_y, 
          batch_size=10,
          epochs=10)

Train on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12392.3727
Epoch 2/10
10000/10000 [==============================] - 14s 1ms/sample - loss: 12332.7983
Epoch 3/10
10000/10000 [==============================] - 16s 2ms/sample - loss: 12362.4786
Epoch 4/10
10000/10000 [==============================] - 15s 2ms/sample - loss: 12381.8192
Epoch 5/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12341.3580
Epoch 6/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12337.6320
Epoch 7/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12358.0993
Epoch 8/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12322.7205
Epoch 9/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12390.4263
Epoch 10/10
10000/10000 [==============================] - 15s 2ms/sample - loss: 12381.1204


In [177]:
model.fit(train_x, train_y, 
          batch_size=10,
          epochs=20)

Train on 10000 samples
Epoch 1/20
10000/10000 [==============================] - 16s 2ms/sample - loss: 12356.6821
Epoch 2/20
10000/10000 [==============================] - 15s 1ms/sample - loss: 12356.3212
Epoch 3/20
10000/10000 [==============================] - 15s 1ms/sample - loss: 12393.1001
Epoch 4/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12219.4047
Epoch 5/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12299.5797
Epoch 6/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12324.0439
Epoch 7/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12323.7565
Epoch 8/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12325.0019
Epoch 9/20
10000/10000 [==============================] - 15s 1ms/sample - loss: 12394.5230
Epoch 10/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12326.8734
Epoch 11/20
10000/10000 [==============================]

In [141]:
from numpy.random import random

In [142]:
train_x = random((100,)).astype(np.float32)
train_y = random((100,)).astype(np.float32)
train_x, train_y

(array([0.46147427, 0.4936299 , 0.7725021 , 0.18668415, 0.19160269,
        0.5329875 , 0.89422435, 0.8431803 , 0.45767853, 0.9006393 ,
        0.30371124, 0.8654656 , 0.64627594, 0.2774724 , 0.34715426,
        0.7767715 , 0.08706174, 0.9893154 , 0.6400852 , 0.9627552 ,
        0.50067353, 0.56687176, 0.48472205, 0.6378169 , 0.6819773 ,
        0.6446982 , 0.13012254, 0.27104077, 0.50481087, 0.25702876,
        0.59998226, 0.45324805, 0.50154966, 0.43757737, 0.66555214,
        0.9135334 , 0.7003318 , 0.23003371, 0.05756523, 0.49323606,
        0.75259066, 0.6157169 , 0.6374104 , 0.5707051 , 0.6783405 ,
        0.89434326, 0.8651092 , 0.9301288 , 0.586712  , 0.7044457 ,
        0.7020338 , 0.09967529, 0.7703017 , 0.85118467, 0.14892976,
        0.24892834, 0.2034809 , 0.26588657, 0.05641429, 0.03270473,
        0.73204887, 0.9724218 , 0.31882647, 0.17593206, 0.7806721 ,
        0.96902066, 0.84500116, 0.7622705 , 0.00976924, 0.772007  ,
        0.31686476, 0.95740134, 0.16487914, 0.99

In [143]:
model = Sequential([
    SimpleRNN(10, input_shape=(10, 1)),
    Dense(1),
])

In [144]:
model.build()

In [145]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 10)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [146]:
model.compile(loss='mse', optimizer='sgd')

In [151]:
train_x.reshape(10, 10, 1)

array([[[0.46147427],
        [0.4936299 ],
        [0.7725021 ],
        [0.18668415],
        [0.19160269],
        [0.5329875 ],
        [0.89422435],
        [0.8431803 ],
        [0.45767853],
        [0.9006393 ]],

       [[0.30371124],
        [0.8654656 ],
        [0.64627594],
        [0.2774724 ],
        [0.34715426],
        [0.7767715 ],
        [0.08706174],
        [0.9893154 ],
        [0.6400852 ],
        [0.9627552 ]],

       [[0.50067353],
        [0.56687176],
        [0.48472205],
        [0.6378169 ],
        [0.6819773 ],
        [0.6446982 ],
        [0.13012254],
        [0.27104077],
        [0.50481087],
        [0.25702876]],

       [[0.59998226],
        [0.45324805],
        [0.50154966],
        [0.43757737],
        [0.66555214],
        [0.9135334 ],
        [0.7003318 ],
        [0.23003371],
        [0.05756523],
        [0.49323606]],

       [[0.75259066],
        [0.6157169 ],
        [0.6374104 ],
        [0.5707051 ],
        [0.6783405 ],
  

In [154]:
model.fit(train_x.reshape(10, 10, 1) * 100, train_y[:10] * 100, 
          batch_size=10,
          epochs=10)

Train on 10 samples
Epoch 1/10
10/10 [==============================] - 0s 1ms/sample - loss: 1693.9922
Epoch 2/10
10/10 [==============================] - 0s 593us/sample - loss: 1245.9900
Epoch 3/10
10/10 [==============================] - 0s 475us/sample - loss: 963.2513
Epoch 4/10
10/10 [==============================] - 0s 502us/sample - loss: 778.3412
Epoch 5/10
10/10 [==============================] - 0s 483us/sample - loss: 666.6307
Epoch 6/10
10/10 [==============================] - 0s 707us/sample - loss: 877.8005
Epoch 7/10
10/10 [==============================] - 0s 611us/sample - loss: 580.0933
Epoch 8/10
10/10 [==============================] - 0s 693us/sample - loss: 1129.6672
Epoch 9/10
10/10 [==============================] - 0s 753us/sample - loss: 948.7717
Epoch 10/10
10/10 [==============================] - 0s 786us/sample - loss: 719.8773
